# Setup of CarpetX

In [ ]:
# %load /usr/local/lib/et-setup.py
# this allows you to use "cd" in cells to change directories instead of requiring "%cd"
%automagic on
# override IPython's default %%bash to not buffer all output
from IPython.core.magic import register_cell_magic
from time import time, sleep
import os
home = os.path.expanduser("~/")
@register_cell_magic
def bash(line, cell):
    get_ipython().system(cell)
@register_cell_magic
def slurm(line, cell):
    with open(f"{home}/.slurm.sh","w") as fd:
        fd.write("if [ ${SLURM_PROCID} != 0 ]; then exit 0; fi\n")
        fd.write(cell)
    get_ipython().system(f"srun {line} bash {home}/.slurm.sh")

# this (non-default package) keeps the end of shell output in view
try: import scrolldown
except ModuleNotFoundError: pass

# We are going to install kuibit, a Python package to post-process Cactus simulations.
# We will install kuibit inside the Cactus directory. The main reason for this is to
# have a make easier to uninstall kuibit (you can just remove the Cactus folder). 
import os, sys
os.environ["PYTHONUSERBASE"] = os.environ['HOME'] + "/Cactus/python"
sys.path.insert(1, f"{os.environ['PYTHONUSERBASE']}/lib/python{sys.version_info[0]}.{sys.version_info[1]}/site-packages")


In [ ]:
cd ~/

In [ ]:
%%bash
# untar.py ~etuser/CarpetX.tar.gz

# NRPy+!
The NRPy+ collection of Python codes is designed to help us write code for Cactus, CarpetX, and other frameworks. Let's check it out...

In [ ]:
%%bash
cd ~/
if [ ! -d nrpytutorial ]; then
  git clone -b ETX_2023_06 --depth 1 https://github.com/zachetienne/nrpytutorial
else
  echo "Already checked out"
fi

In [ ]:
cd ~/nrpytutorial

## Writing a Thorn a New Way...
Instead of writing a ton of boilerplate files, we can construct thorns with loops that evaluate fuctions as easy as sympy.

In [ ]:
%%writefile WaveToy.py
import os
import sys
import re
import grid
from cactusthorn import CactusThorn, loop
from sympy import sympify, sin, cos, pi
import NRPy_param_funcs as par
from subprocess import call
import numpy as np
from fstr import f

def main():
    global par

    # Current options are Carpet and CarpetX
    grid.ET_driver = os.environ.get("CACTUS_DRIVER","Carpet")

    thorn = CactusThorn("TestOne","WaveToyNRPy")

    #FD_order = thorn.declare_param('FD_order',default=2,vmin=2,vmax=8,doc="The finite difference order")

    wave_speed = thorn.declare_param('wave_speed',default=1,vmin=.1,vmax=100,doc="The speed of the wave")
    x0 = thorn.declare_param('x0',default=0,vmin=-100,vmax=100,doc="The x pos of the wave")
    y0 = thorn.declare_param('y0',default=0,vmin=-100,vmax=100,doc="The y pos of the wave")
    z0 = thorn.declare_param('z0',default=0,vmin=-100,vmax=100,doc="The z pos of the wave")
    zero = thorn.declare_param('zero',default=0,vmin=0,vmax=0,doc="zero")

    # Centering for X, Y, and Z
    centering='VVC'

    # AUXEVOL needed for the evo, can be freed after evaluating rhs (1 time level)
    # AUX uu_rhs (1 time level)
    # EVOL evolved gfs (3 time levels)
    uu_rhs, vv_rhs = thorn.register_gridfunctions("AUX", ["rhs_uu", "rhs_vv"], centering=centering)
    uu, vv = thorn.register_gridfunctions("EVOL", ["uu", "vv"], centering=centering)
    ana = thorn.register_gridfunctions("AUX", ["ana"], centering=centering)
    cctk_time = par.Cparameters("CCTK_REAL","Cactus",["cctk_time"],0)
    if grid.ET_driver == "CarpetX":
        tmp0, tmp1 = thorn.register_gridfunctions("TILE_TMP",["tmp0v","tmp1v"],centering=centering)
        regrid_error = thorn.get_regrid_error()
    x,y,z = thorn.get_xyz()

    from outputC import lhrh
    import indexedexp as ixp
    import NRPy_param_funcs as par

    FD_order = 2
    par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER",FD_order)

    uu_dDD = ixp.declarerank2("uu_dDD","sym01")

    if grid.ET_driver == "CarpetX":
        evol_eqns = [
            lhrh(lhs=tmp0, rhs=uu_dDD[0][0]),
            lhrh(lhs=tmp1, rhs=uu_dDD[1][1]),
            loop,
            lhrh(lhs=uu_rhs, rhs=vv),
            lhrh(lhs=vv_rhs, rhs=wave_speed**2*(tmp0 + tmp1))
        ]
    else:
        # Version of evolution equations without temporaries
        evol_eqns = [
            lhrh(lhs=uu_rhs, rhs=vv),
            lhrh(lhs=vv_rhs, rhs=wave_speed**2*(uu_dDD[0][0] + uu_dDD[1][1]))
        ]

    k = sympify(pi/20)
    toff = sympify(pi/2)
    sq2 = sympify(2**.5)

    init_eqns = [
        lhrh(lhs=vv, rhs=sympify(0)),
        lhrh(lhs=uu, rhs=sin(k*(x))*sin(k*(y)))
    ]

    # What is omega?
    # omega / k = wave_speed

    anal_eqns = [
        lhrh(lhs=ana, rhs=sin(k*x)*sin(k*y)*sin(cctk_time*sq2*wave_speed*k+toff)-uu)
    ]

    if grid.ET_driver == "CarpetX":
        thorn.add_func("refine",
            body=[lhrh(lhs=regrid_error, rhs=10/((x-20)**2 + (y-20)**2))],
            doc="do the regrid",
            schedule_bin="ODESolvers_EstimateError")

    # access a variable with a different centering using interpolation
    # looping cell-centered, access vertex-centered, not vice-versa
    # all rhs variables should have the same centering
    # wave toy with fluxes, fluxes are faces
    # schedule something in post-regrid, apply bc's
    thorn.add_func("wave_init",
        body=init_eqns,
        where='everywhere',
        schedule_bin='initial',
        doc='Do the wave init',
        centering=centering)

    thorn.add_func("wave_evol",
        body=evol_eqns,
        where='interior',
        schedule_bin='RHS',
        doc='Do the wave evol',
        centering=centering)

    thorn.add_func("wave_anal",
        body=anal_eqns,
        where='everywhere',
        schedule_bin='Analysis',
        doc='Check the result',
        centering=centering)

    assert "CACTUS_HOME" in os.environ, "Please set the CACTUS_HOME variable to point to your Cactus installation"
    cactus_home = os.environ["CACTUS_HOME"]
    cactus_sim = os.environ.get("CACTUS_SIM","sim")
    cactus_thornlist = os.environ.get("CACTUS_THORNLIST", None)

    #cactus_home = "/project/sbrandt/release/Cactus"
    thorn.generate(cactus_home,cactus_config=cactus_sim,cactus_thornlist=cactus_thornlist)

main()

In [ ]:
%%bash
export PYTHONPATH=$(pwd)
export CACTUS_HOME=$HOME/CarpetX
export CACTUS_DRIVER=CarpetX
export CACTUS_THORNLIST=$HOME/asterx.th
python3 WaveToy.py

In [ ]:
%%bash
diff ~/asterx.th ~etuser/asterx.th

In [ ]:
%%slurm -n 2
cd ~/CarpetX
./simfactory/bin/sim build -j${SLURM_NPROCS} --thornlist ~/asterx.th --optionlist ~etuser/carpetx.cfg

In [ ]:
%%writefile wave2dx.par
# This is the parameter file which will tell Cactus
# which thorns to execute and with what values
ActiveThorns = "WaveToyNRPy CarpetX IOUtil ODESolvers"

ODESolvers::method = "RK3"
Cactus::presync_mode = "presync-only"

Cactus::terminate = "iteration"
$blocksize=10
$nblocks=20
Cactus::cctk_itlast = 50 #$nblocks*$blocksize

CarpetX::periodic_x = false
CarpetX::periodic_y = false
CarpetX::periodic_z = false

CarpetX::max_tile_size_x = 200
CarpetX::max_tile_size_y = 200
CarpetX::max_tile_size_z = 200
CarpetX::max_grid_size_x = 200
CarpetX::max_grid_size_y = 200
CarpetX::max_grid_size_z = 200

CarpetX::verbose = yes

CarpetX::poison_undefined_values = no

$ncells_x = 80
$ncells_y = 80
$xmax = 40.0
$ymax = 40.0
$dx = $xmax/($ncells_x + 2.0)
$dy = $ymax/($ncells_y + 2.0)

CarpetX::ncells_x = $ncells_x
CarpetX::ncells_y = $ncells_y
CarpetX::ncells_z = 1

CarpetX::xmin = $dx
CarpetX::ymin = $dy
CarpetX::zmin = -1.0

CarpetX::xmax = $xmax-$dx
CarpetX::ymax = $ymax-$dy
CarpetX::zmax = 1.0

CarpetX::blocking_factor_x = 8
CarpetX::blocking_factor_y = 8
CarpetX::blocking_factor_z = 1

CarpetX::ghost_size_x = 1
CarpetX::ghost_size_y = 1
CarpetX::ghost_size_z = 0

IO::out_dir = $parfile
IO::out_every = 2 #$blocksize
IO::out_mode = "np"
IO::out_proc_every = 1
CarpetX::out_openpmd_vars = "all"
CarpetX::boundary_x = "dirichlet"
CarpetX::boundary_y = "dirichlet"
CarpetX::boundary_z = "dirichlet"
CarpetX::boundary_upper_x = "dirichlet"
CarpetX::boundary_upper_y = "dirichlet"
CarpetX::boundary_upper_z = "dirichlet"

$nlevels = 2
CarpetX::max_num_levels = $nlevels
CarpetX::regrid_every = 1
CarpetX::regrid_error_threshold = 0.1
CarpetX::out_tsv_vars = " WaveToyNRPy::uugf "

In [ ]:
%%bash
cd ~/CarpetX
./simfactory/bin/sim create-run --procs=2 --parfile ~/nrpytutorial/wave2dx.par wave2dx

In [ ]:
ls -d ~/simulations/wave2dx/output-0000/wave2dx/*.bp5

In [ ]:
%%bash
curl -o ~/plot-zslice.py -L https://gist.githubusercontent.com/stevenrbrandt/ef3718d52680d8a4e6accbad198a0b42/raw/1fcfd880ec426255c8678f8dd10a701b456f5c8a/plot-zslice.py

In [ ]:
%%bash
export PYTHONPATH=/usr/local/lib/python3.8/site-packages
rm -f *.png
python3 ~/plot-zslice.py --vmin -.1 --vmax .1 \
  --out-dir . --data-dir $HOME/simulations/wave2dx/output-0000/wave2dx WaveToyNRPy::uuGF

In [ ]:
%%bash
ls *.png

In [ ]:
from IPython.display import Image, display
display(Image(filename=('uugf%05d.png' % 26)))

In [ ]:
from subprocess import call
call(["ffmpeg", "-i", "uugf%05d.png", "-i", "uugf00001.png",
      "-filter_complex", "[1:v] palettegen [p];[0:v][p] paletteuse",
      "-y", "wave2d.gif"])

In [ ]:
display(Image("wave2d.gif"))